<img src="CIDaeNNB.png" alt="Logo CiDAEN" align="right">


<br><br><br>
<h1><font color="#00586D" size=5>Módulo 1</font></h1>



<h1><font color="#00586D" size=6>Python - Avanzado</font></h1>

<br>
<div style="text-align: right">
<font color="#00586D" size=3>Javier Cózar</font><br>
<font color="#00586D" size=3>Máster en Ciencia de Datos y Desarrollo de Aplicaciones en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>

<h2><font color="#00586D" size=5>Índice</font></h2>

---

El objetivo de este tutorial es aspectos más específicos de python como los ámbitos, así como funcionalidades habituales en cualquier lenguaje de programación (paquetes, ficheros y clases).


A modo de **índice**, en este tutorial se verán:


1. Eficiencia sobre una lista de elementos
  1. Inicialización de lista
  1. Iteradores y generadores: ¿Qué son?
  1. Comprehension lists
2. Manejo de excepciones
3. Orden superior
  1. Funciones lambda
  1. Funciones filter, map y reduce

---

<h1><font color="#00586D" size=5>1. Eficiencia sobre una lista de elementos </font></h1>

Cuando trabajemos con conjuntos de datos, sobre todo cuando éstos son especialmente grandes, es muy importante tener en cuenta la eficiencia.

En esta sección abordaremos detalles de implementación que tienen un gran impacto en el rendimiento de nuestro código.

<h1><font color="#00586D" size=4>1.1 Inicialización de listas </font></h1>

Como hemos visto, no es necesario especificar el tamaño de las listas en su creación (al contrario que en otros lenguajes de proogramación). Esto, aunque muy util, conlleva una penalización en cuanto al uso de memoria RAM. Además, exite un sobrecoste en tiempo de ejecución cuando hay que expandir el tamaño de la lista (uso de la función `append`).

In [1]:
%%timeit -n 10
# Uso de append (va aumentando el tamaño de la lista dinámicamente)

l = []
N = 10**6
for i in range(N):
    l.append(i)

139 ms ± 9.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
%%timeit -n 10
# Inicializamos la lista a un tamaño N (usando elementos None)

N = 10**6
l = [0] * N
for i in range(N):
    l[i] = i

89.4 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<font size=4> <i class="fa fa-book" aria-hidden="true" style="color:#00586D"></i> </font> __Importante!__ Por ello, si se conoce el tamaño de la lista a priori, es mejor generar una lista con todos los elementos (inicialmente `None`, `0`, ...) y hacer asignaciones mediante el índice.

Aún más eficiente es el uso de **comprehension lists** (que veremos a continuación).

<h1><font color="#00586D" size=4>1.2 Iteradores y generadores: ¿Qué son? </font></h1>

Conceptualmente, un iterador es cualquier objeto que se puede iterar, es decir, recorrer sus elementos mediante un bucle o equivalente (acaba llamando a una función interna `__iter__`).

Un generador es un objeto que genera un elemento cada vez que se le solicita (a trav'es de una llamada a una función interna `__next__`). Todo generador es un iterador, pero no todo iterador es un generador: por ejemplo, la función `enumerate` devuelve un generador (e iterador) pero la función `range` devuelve un iterador (pero no generador).

<div class="alert alert-block alert-info">
<i class="fa fa-info-circle" aria-hidden="true"></i>
La principal diferencia radica en la implementación, siendo mucho más legible y fácil de implementar un generador (estructura de función) que un iterador (estructura de clase).
</div>

In [ ]:
# Al constructor de una lista se le puede pasar un objeto iterable, inicializando la lista con todos los elementos del mismo.
# Una lista también es un iterable. Ejemplo:
l = list(range(10))
print(l)

# Pero también se le puede pasar otra lista!
l = list([1,2,3])
print(l)

La diferencia entre manejar un iterable y la propia lista de elementos radica en la eficiencia: mientras que la **lista de elementos debe permanecer alojada (por completo) en memoria principal** (habiéndose generado inicialmente por completo), el **iterable puede ir creando los objetos dinámicamente uno a uno**. Ejemplo usando `range`:

<font color="#00586D" size=3> Range</font>

In [12]:
%%timeit -n 10
N = 10 ** 6
suma = 0
for _ in list(range(N)):  # Creando la lista y luego iterándola
    suma += 1

105 ms ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit  -n 10
N = 10 ** 6
suma = 0
for _ in range(N):  # Iterando el objeto range (iterable) directamente
    suma += 1

65.2 ms ± 4.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<font size=4> <i class="fa fa-book" aria-hidden="true" style="color:#00586D"></i> </font> __Importante!__ Lo realmente costoso no está en el bucle (en ambos casos el bucle itera N veces la operación +=1). Lo costoso está en la creación de la lista entera en memoria, con el consecuente consumo de memoria para alojarla.

In [14]:
%%timeit
N = 10 ** 6
l = list(range(N))

48 ms ± 5.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit
N = 10 ** 6
r = range(N)

352 ns ± 47.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


<font color="#00586D" size=3> Enumerate</font>

En ejemplos anteriores hemos iterado sobre los elementos de una lista (`for elemento in lista`) y sobre los índices (`for i in range(len(lista))`). Este último caso se da cuando necesitamos el índice para, por ejemplo, actualizar la posición de la lista.

Es posible iterar sobre los elementos y los índices simultáneamente haciendo uso de `enumerate` (un generador):

In [16]:
lista = [1, 2, 3, 4, 5]

for (i, elemento) in enumerate(lista):
    lista[i] = elemento**2

print(lista)

[1, 4, 9, 16, 25]


In [ ]:
lista = [1, 2, 3, 4, 5]

for i in range(len(lista)):
    lista[i] = lista[i]**2

print(lista)

Lo que se le pasa a `enumerate` es un iterable, y lo que hace es añadir el índice, de tal manera que por cada elemento devuelve una tupla (índice, elemento). El elemento puede ser a su vez otra tupla, o cualquier otro objeto de python:

In [ ]:
tuplas = [(1, 2), (3, 4), (5, 6)]

for i, tupla in enumerate(tuplas):
    # Nótese que en cada iteración la lista tuplas va cambiando!
    # En la siguiente iteración, se actualizará la posición i con el valor modificado en la iteración anterior
    tuplas[i] = (tuplas[i-1][0] + tupla[0], tuplas[i-1][1] + tupla[1])

print(tuplas)

<font color="#00586D" size=3> Zip</font>

Otra función que suele ser de utilizad es la función `zip`. Esta función permite iterar simultáneamente dos iterables:

In [19]:
lista1 = [1, 2, 3, 4, 5]
lista2 = range(6,8)
suma = []

for e1, e2 in zip(lista1, lista2):
    suma.append(e1 + e2)

print(suma)

[7, 9]


Si los iterables son de **diferente tamaño**, se iterará sobre N elementos, siendo N el **mínimo de los dos tamaños**.

In [ ]:
lista1 = range(1,6)
lista2 = range(6,9)
suma = []

for e1, e2 in zip(lista1, lista2):
    suma.append(e1 + e2)

print(suma)

Siguiendo la filosofía de la eficiencia, se puede combinar enumerate y zip para no tener que generar ninguna lista en memoria, y además evitar el uso de append en la lista resultado (suma)

Sin embargo, llamar a las funciónes también incrementa el tiempo de ejecución

In [20]:
%%timeit -n 10

N = 10 ** 6
lista1 = range(0,N)
lista2 = range(0,N)

suma = []

for e1, e2 in zip(lista1, lista2):
    suma.append(e1 + e2)

211 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%%timeit -n 10
# Siguiendo la filosofía de la eficiencia, se puede combinar enumerate y zip para no tener que 
# generar ninguna lista en memoria, y además evitar el uso de append en la lista resultado (suma)
N = 10 ** 6
lista1 = range(0,N)
lista2 = range(0,N)

suma = [None] * N

for i, (e1, e2) in enumerate(zip(lista1, lista2)):
    suma[i] = e1 + e2

242 ms ± 34.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<h1><font color="#00586D" size=4>1.3 Comprehension lists </font></h1>

En los ejemplos anteriores hemos creado inicialmente una lista de resultados (suma), y le asignamos los valores en un bucle posterior. Esto significa que:

* Usamos append para ir añadiendo los valores, uno a uno, a la lista
* Gastamos tiempo en crear la lista en memoria (con valores None por ejemplo), y posteriormente asignamos los valores

Existe una alternativa más eficiente: los **comprehension lists**

Permiten generar una lista con unos valores en un solo paso, con la ventaja de que el código implícito está optimizado **(mucho más eficiente)**. En este sentido, se pueden crear listas, generadores y diccionarios.

<font color="#00586D" size=3> Listas</font>

In [22]:
%%timeit -n 10

N = 10 ** 6
lista1 = range(0,N)
lista2 = range(0,N)

suma = [None] * N

for (i, (e1, e2)) in enumerate(zip(lista1, lista2)):
    suma[i] = e1 + e2

233 ms ± 63.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%%timeit -n 10

N = 10 ** 6
lista1 = range(0,N)
lista2 = range(0,N)

suma = [ e1 + e2 for e1, e2 in zip(lista1, lista2) ]

136 ms ± 5.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Se pueden combinar expresiones condicionales en la comprehension list, te tal manera que solo cuando la expresión se evalua a True se genera el elemento correspondiente **(permitiendo que la lista resultante sea más pequeña que la original)**

In [24]:
N = 100
lista1 = range(0,N)
lista2 = range(0,N)

suma_impares = [ e1 + e2 for e1, e2 in zip(lista1, lista2) if e1 % 2 == 1 and e2 % 2 == 1]
print(suma_impares)

[2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62, 66, 70, 74, 78, 82, 86, 90, 94, 98, 102, 106, 110, 114, 118, 122, 126, 130, 134, 138, 142, 146, 150, 154, 158, 162, 166, 170, 174, 178, 182, 186, 190, 194, 198]


In [32]:
[i for lista_indices in [[1, 2, 3], [2, 5, 2]] for i in lista_indices]

[1, 2, 3, 2, 5, 2]

<font color="#00586D" size=3> Expresiones generadoras</font>

Esta sintáxis también se puede utilizar para crear generadores:

In [37]:
N = 10 ** 6
lista1 = range(0,N)
lista2 = range(0,N)

suma = ( e1 + e2 for e1, e2 in zip(lista1, lista2) )

In [38]:
suma

<generator object <genexpr> at 0x113187bd0>

In [39]:
N = 10
lista1 = range(0,N)
lista2 = range(0,N)

suma = list([ e1 + e2 for e1, e2 in zip(lista1, lista2) ])

In [40]:
suma

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

<h3><font color="#00586D" size=4><i class="fa fa-pencil-square-o" aria-hidden="true"></i>Ejercicio</font></h3>

Crea una comprehension list que contenga los números múltiplos de 5 menores que 100.

<h3><font color="#00586D" size=4><i class="fa fa-pencil-square-o" aria-hidden="true"></i>Ejercicio</font></h3>

Crea una comprehension list que multiplique los números del 1 al 100 con los números del 100 al 1, es decir:

```python
[1 * 100, 2 * 99, 3 * 98, ..., 99 * 2, 100 * 1]
```

¿Cuál es la media de los números en la lista?

<font color="#00586D" size=3> Diccionarios (comprehension dicts)</font>

In [41]:
N = 10
lista1 = range(0,N)
lista2 = range(0,N)

suma = {(e1, e2):  e1 + e2 for e1, e2 in zip(lista1, lista2)}

In [42]:
suma

{(0, 0): 0,
 (1, 1): 2,
 (2, 2): 4,
 (3, 3): 6,
 (4, 4): 8,
 (5, 5): 10,
 (6, 6): 12,
 (7, 7): 14,
 (8, 8): 16,
 (9, 9): 18}

<h3><font color="#00586D" size=4><i class="fa fa-pencil-square-o" aria-hidden="true"></i>Ejercicio</font></h3>

Usar la función `es_primo`para generar una diccionario que contenga como clave los números del 1 al 100 y como valor un booleano indicando si es primo o no.

In [ ]:
def es_primo(n):
    return not any( (n % d == 0 for d in range(2, n // 2 + 1)) )

<font color="#00586D" size=3> Conjuntos (comprehension sets)</font>

In [43]:
N = 10
lista = range(0,N)

multiplos3 = {e for e in lista if e % 3 == 0}

In [44]:
multiplos3

{0, 3, 6, 9}

<h3><font color="#00586D" size=4><i class="fa fa-pencil-square-o" aria-hidden="true"></i>Ejercicio</font></h3>

Usar la función `es_primo`anterior para generar un conjunto que contenga los números primos menores que 100.

<h1><font color="#00586D" size=5>2. Manejo de excepciones</font></h1>

Hay ocasiones en las que sabemos que el código puede fallar, pero queremos gestionar esa excepción de forma controlada. Por ejemplo, una conexión a un web socket puede fallar por problemas en la red.

Para gestionar las excepciones, se debe situar el código que puede lanzar esta excepción entre las palabras `try` y `except`. En la parte de `except` se introduce el código para tratar dicha excepción.

In [47]:
try:
    numerador = 10
    denominador = int(input("Introduce el denominador"))
    print(f"{numerador} / {denominador} = {numerador / denominador}")
except:
    print("No se puede dividir por 0!")

Introduce el denominador a


No se puede dividir por 0!


En la parte del except se puede, además, especificar el tipo de error esperado y utilizar más de un bloque `except`. Es decir, se puede gestionar más de un posible error en el código `try`.

In [53]:
int("2.3")

ValueError: invalid literal for int() with base 10: '2.3'

In [61]:
mi_excepcion = None
try:
    numerador = 10
    denominador = int(input("Introduce el denominador"))
    print(f"{numerador} / {denominador} = {numerador / denominador}")
except ZeroDivisionError:
    print("No se puede dividir por 0!")
except ValueError as ve:
    mi_excepcion = ve
    print(f"Debes introducir un número como denominador! Error: {ve}")

Introduce el denominador 0


No se puede dividir por 0!


También podemos lanzar una excepción por código como sistema de control de errores. Por ejemplo, si solicitamos por teclado un entero entre 1 y 10, podemos lanzar una excepción si la condición no se cumple. Podemos crear nuestras propias excepciones construyendo una nueva clase que herede de una excepción, aunque no llegaremos a utilizar esta técnica a lo largo del máster.

In [63]:
try:
    x = int(input("Introduce un número entero entre 1 y 10"))
except ValueError as ve:
    print("Debes introducir un número entero")
if not (1 <= x <= 10):
    raise Exception("El número debe estar comprendido entre 1 y 10")

Introduce un número entero entre 1 y 10 15


Exception: El número debe estar comprendido entre 1 y 10

<h3><font color="#00586D" size=4><i class="fa fa-pencil-square-o" aria-hidden="true"></i>Ejercicio</font></h3>

Solicita un valor por teclado, e imprime el siguiente mensaje en función del valor utilizando la gestión de excepciones:

- "Soy entero" si el valor es un entero.
- "Soy flotante" si el valor es un flotante.
- "Soy una letra" si el valor es un character.
- "Soy una palabra" si el valor es una secuencia de letras sin espacios.
- "Soy una frase" en caso contrario.

<h1><font color="#00586D" size=5>3. Orden superior</font></h1>

Ya vimos cómo declarar funciones. Los argumentos podían ser opcionales o requeridos. Además, estos argumentos pueden ser de cualquier tipo (enteros, listas, diccionarios, e incluso otras funciones!). Utilizar funciones como argumentos de otras funciones, además de poder devolver como resultado otra nueva función es lo que se denomina **orden superior**.

A continuación vamos a declarar una función genérica que aplica operaciones a dos argumentos numéricos. Después usaremos esta función proporcionándole otra función que aplique una operación concreta.

In [65]:
a = 10
b = 20

def opera(x, y, fn):
    return fn(x, y)

def suma(x, y):
    return x + y

def mult(x, y):
    return x * y

c = opera(a, b, mult)

print(f"{a} operado con {b} = {c}")

10 operado con 20 = 200


<h3><font color="#00586D" size=4><i class="fa fa-pencil-square-o" aria-hidden="true"></i>Ejercicio</font></h3>

Como hemos visto en las celdas de tipo Markdown, el texto en negrita se especifica mediante un texto encerrado entre dos dobles asteriscos. Si queremos que sea en cursiva, se debe encerrar entre dos símbolos `_`. Implementar:

- Una función `negrita` que reciba un texto y devuelva el mismo encerrado entre dobles asteriscos.
- Una función `cursiva` que reciba un texto y devuelva el mismo encerrado entre barras bajas.
- Una función `selector` que pida por teclado que se introduzca una de las palabras `negrita` o `cursiva`. Deve devolver una de las dos funciones anteriores en función del valor introducido.

A continuación:

1. Solicitar un texto por teclaro.
2. Crear una función que reciba como argumento un texto y la función `selector`. Ésta debe llamar a la función (sin argumentos), y devolver el texto decorado en negrita o cursiva.
3. Invocar la función anterior.

_Texto_

<h1><font color="#00586D" size=4> 3.1 Funciones lambda</font></h1>

Además de la definición estándar de funciones existe la posibilidad de crar funciones en una sola línea. Estas funciones se llaman `lambda`, y son funciones que ejecutan una expresión y devuelve ese resultado.

In [ ]:
# Ejemplo de función lambda
# Usando la expresión de potencia **, crear la función exp (e ** x)
import math
exp = lambda x: math.e ** x
n = 5.7
print("La exponencial de {:f} es: {:f}".format(n, exp(n)))

La ventaja del uso de estas funciones radica en la legibilidad a la hora de proporcionar funciones como argumentos, permitiendo declararlas en una línea sin necesidad de asignarla a una variable (por ello se llaman funciones anónimas): 

In [68]:
a = 10
b = 20

# Declarando la función
def opera(x, y, fn):
    return fn(x, y)

print(f"{a} operado con {b} = {opera(a, b, lambda x, y: x / y)}")

10 operado con 20 = 0.5


In [71]:
from collections import defaultdict
import requests  # instalar si no está instalado
url_quijote = "https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt"
text_quijote = requests.get(url_quijote).text

palabras = defaultdict(int)
for palabra in text_quijote.split():
    palabras[palabra.lower()] += 1 

sorted( ((palabra, count) for palabra, count in palabras.items()), key=lambda x: x[1], reverse=True)[:20]

[('que', 10384),
 ('de', 9026),
 ('y', 8449),
 ('la', 5009),
 ('a', 4806),
 ('en', 4030),
 ('el', 3842),
 ('no', 2910),
 ('se', 2382),
 ('los', 2147),
 ('con', 2078),
 ('por', 1910),
 ('su', 1861),
 ('le', 1802),
 ('lo', 1801),
 ('las', 1488),
 ('me', 1155),
 ('como', 1140),
 ('del', 1127),
 ('don', 1065)]

<h1><font color="#00586D" size=4> 3.2 Funciones filter, map y reduce</font></h1>

Estas funciones permiten aplicar una función sobre los elementos de un iterable. Las tres funciones tienen dos argumentos: el primero es la función a aplicar, y el segundo es el objeto iterable sobre el que aplicarlos.

* La función `filter` filtra los elementos del iterable en base al resultado una función que devuelve un booleano con cada elemento de la lista. Si es `True` permanece en el resultado, y si es `False` lo filtra.

* La función `map` permite aplicar una función a cada uno de los elementos, generando un segundo iterable.

* La función `reduce` permite combinar todos los elementos de la lista aplicando una finción, dos a dos iterativamente, hasta reducir a un solo elemento.

#### <font color="#004D7F" size=3>  Filter</font>


In [72]:
# Filtrar los numeros impares
l = range(10)
def es_par(x):
    return x % 2 == 0
pares = filter(es_par, l)
print(pares)
print(list(pares))

[0, 2, 4, 6, 8]


In [73]:
# ¡En este caso es mucho más cómodo usar funciones lambda!
l = range(10)
pares = filter(lambda x: x%2==0, l)
print(pares)
print(list(pares))

[0, 2, 4, 6, 8]


#### <font color="#004D7F" size=3> Map</font>


In [76]:
l = range(10)
def cuadrado(x):
    return x**2
cuadrados = map(cuadrado, l)
print(cuadrados)
print(list(cuadrados))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [78]:
# Tanto las funciones filter, como map y reduce esperan funciones con 1 único parámetro (en el caso de filter y map)
# o 2 parámetros (función reduce). 
# Si queremos aplicar una función con más parámetros, pero especificando un valor específico para un un subconjunto
# de ellos (todos menos uno) se puede hacer mediante el uso de funciones lambda

def suma(x, y):
    return x + y
l = range(10)
# En este ejemplo queremos aplicar la función suma (2 parámetros), fijando uno de ellos a 10.
cuadrados = map(lambda x: suma(x, 10), l)
print(list(cuadrados))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


#### <font color="#004D7F" size=3> Reduce</font>


Aunque en python 3.0 la función `reduce` se [eliminó](https://docs.python.org/3.0/whatsnew/3.0.html#builtins) como función `built-in` (y se movió a un módulo, llamado `functools`), es útil que la veamos porque será de utilizad en posteriores módulos (al menos conceptualmente).

La razón por la que la movieron, según la documentación oficial, es:

_"Removed reduce(). Use functools.reduce() if you really need it; however, 99 percent of the time an explicit for loop is more readable."_

Sin embargo, aunque puede resultar más dificil de leer, requiere menos código (evitamos el uso de bucles), y desde el punto de vista de la _programación funcional_ es más **intuitiva**.

In [79]:
import functools
l = range(10)
suma = functools.reduce(lambda x, y: x+y, l)
print(suma)

45


In [ ]:
no_meaning = ["el", "la", "le", "lo", "las", "los", "que", "de", "y", "a", "en", "los", "con", "por", "su", "me", "del", "como"]

<h3><font color="#00586D" size=4><i class="fa fa-pencil-square-o" aria-hidden="true"></i>Ejercicio</font></h3>

Usar las funciones filter, map y reduce para:

Crear una lista de palabras de `text_quijote` como `palabras = text_quijote.split()`

1. Transformar todas las palabras de `palabras` a minúsculas y eliminar el punto final de aquellas palabras que lo tengan
2. Descartar todas las palabras en la lista de palabras `no_meaning`
3. Obtener el número de caracteres de cada palabra
4. Obtener la suma de caracteres de todas las palabras, y calcular el porcentaje de caracteres que se han descartado en los dos primeros pasos.